In [1]:
import os
import re
import nltk
import pandas as pd
import csv
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [3]:
%cd C:\Users\Vedant\Desktop\NLP LAB\Assignment9\english_dataset

C:\Users\Vedant\Desktop\NLP LAB\Assignment9\english_dataset


In [42]:
dataset = pd.read_csv('english_dataset.tsv',delimiter='\t',encoding='utf-8')
df_train = list(dataset['text'])
dataset_test = pd.read_csv('hasoc2019_en_test-2919.tsv',delimiter='\t',encoding='utf-8')
df_test = list(dataset_test['text'])

In [43]:
dataset1 = dataset[dataset['task_1'] == 'HOF']
dataset_test1 = dataset_test[dataset_test['task_1'] == 'HOF']
df_trains = {0:df_train,1:list(dataset1['text']),2:list(dataset1['text'])}
df_tests = {0:df_test,1:list(dataset_test1['text']),2:list(dataset_test1['text'])}
targets = {0:dataset['task_1'],1:dataset1['task_2'],2:dataset1['task_3']}
targets_test = {0:dataset_test['task_1'],1:dataset_test1['task_2'],2:dataset_test1['task_3']}

Preprocessing

In [6]:
from bs4 import BeautifulSoup
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import unidecode
from word2number import w2n
from pycontractions import Contractions
import gensim.downloader as api
from nltk.tokenize import TweetTokenizer
import string
import re

model = api.load("glove-twitter-25")
cont = Contractions(kv_model=model)
cont.load_models()
stop = stopwords.words('english')


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def remove_whitespace(text):
    text = text.strip()
    return " ".join(text.split())

def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text

def expand_contractions(text):
    text = list(cont.expand_texts([text], precise=True))[0]
    return text

def text_preprocessing(text, accented_chars=True, contractions=True, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=True,
                       remove_html=True, remove_num=True, special_chars=True, 
                       stop_words=True):
    
    if remove_html == True: #remove html tags
        text = strip_html_tags(text)
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)
    #if convert_num == True:
    #    text = 
    #if special_chars == True:
    #    text = 
    if punctuations == True:
        text = re.sub('[^a-zA-Z0-9 \n\.]', '', text)
    if remove_num == True:
        text = re.sub(r'\d', '',text)
    if accented_chars == True: #remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()

    tknzr = TweetTokenizer()
    doc = tknzr.tokenize(text) #tokenise text
    
    if stop_words == True:
        for w in doc:
            if w in stop:
                doc.remove(w)
    if lemmatization == True:
        wnl = WordNetLemmatizer()
        doc = [wnl.lemmatize(i) for i in doc] 
    return doc

In [10]:
Vocab = []
for text in df_train:
    for x in text.split(" "):
        Vocab.append(x)
Vocab = list(set(Vocab))

In [22]:
for j in range(3):
    for i in range(len(df_trains[j])):
        df_trains[j][i] = text_preprocessing(df_trains[j][i])
        df_trains[j][i] = " ".join(df_trains[j][i])
    for i in range(len(df_tests[j])):
        df_tests[j][i] = text_preprocessing(df_tests[j][i])
        df_tests[j][i] = " ".join(df_tests[j][i])

In [23]:
count_vect = CountVectorizer(analyzer = 'word',vocabulary = Vocab)
tfidf_transformer = TfidfTransformer()

Model1

In [24]:
for i in range(3):
    X_train_counts = count_vect.fit_transform(df_trains[i])
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    X_test_counts = count_vect.fit_transform(df_tests[i])
    X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
    le = LabelEncoder()
    le.fit(list(targets[i]))
    targets[i] = le.transform(targets[i].astype(str))
    clf = SGDClassifier().fit(X_train_tfidf, targets[i])
    predicted = clf.predict(X_test_tfidf)
    targets_test[i] = le.transform(targets_test[i].astype(str))
    print(classification_report(targets_test[i], predicted))

              precision    recall  f1-score   support

           0       0.40      0.66      0.50       288
           1       0.86      0.66      0.75       865

    accuracy                           0.66      1153
   macro avg       0.63      0.66      0.62      1153
weighted avg       0.74      0.66      0.68      1153

              precision    recall  f1-score   support

           0       0.56      0.67      0.61       124
           1       0.27      0.10      0.14        71
           2       0.51      0.63      0.57        93

    accuracy                           0.52       288
   macro avg       0.45      0.47      0.44       288
weighted avg       0.48      0.52      0.48       288

              precision    recall  f1-score   support

           0       0.85      0.96      0.90       245
           1       0.15      0.05      0.07        43

    accuracy                           0.82       288
   macro avg       0.50      0.50      0.49       288
weighted avg       0

Model2

In [33]:
for i in range(3):
    X_train_counts = count_vect.fit_transform(df_trains[i])
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    X_test_counts = count_vect.fit_transform(df_tests[i])
    X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
    le = LabelEncoder()
    le.fit(list(targets[i]))
    targets[i] = le.transform(targets[i].astype(str))
    clf = LinearSVC().fit(X_train_tfidf, targets[i])
    predicted = clf.predict(X_test_tfidf)
    targets_test[i] = le.transform(targets_test[i].astype(str))
    print(classification_report(targets_test[i], predicted))

              precision    recall  f1-score   support

           0       0.39      0.61      0.48       288
           1       0.84      0.69      0.76       865

    accuracy                           0.67      1153
   macro avg       0.62      0.65      0.62      1153
weighted avg       0.73      0.67      0.69      1153

              precision    recall  f1-score   support

           0       0.55      0.74      0.63       124
           1       0.41      0.10      0.16        71
           2       0.53      0.59      0.56        93

    accuracy                           0.53       288
   macro avg       0.50      0.48      0.45       288
weighted avg       0.51      0.53      0.49       288

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0

Using Pretrained Embeddings : GLOVE

In [34]:
embeddings_dict = {}

In [35]:
with open("C:/Users/Vedant/Desktop/NLP LAB/Assignment9/glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [36]:
len(embeddings_dict)

400000

In [44]:
for j in range(3):
    for i in range(len(df_trains[j])):
        df_trains[j][i] = text_preprocessing(df_trains[j][i])
        e = np.zeros((1,100))
        for word in df_trains[j][i]:
            try:
                e = e + embeddings_dict[word]
            except:
                pass
        df_trains[j][i] = e/len(df_trains[j][i])
    for i in range(len(df_tests[j])):
        df_tests[j][i] = text_preprocessing(df_tests[j][i])
        e = np.zeros((1,100))
        for word in df_tests[j][i]:
            try:
                e = e + embeddings_dict[word]
            except:
                pass
        df_tests[j][i] = e/len(df_tests[j][i])

In [46]:
df_trains[0] = np.array(df_trains[0]).reshape(-1,100)
df_trains[1] = np.array(df_trains[1]).reshape(-1,100)
df_trains[2] = np.array(df_trains[2]).reshape(-1,100)
df_tests[0] = np.array(df_tests[0]).reshape(-1,100)
df_tests[1] = np.array(df_tests[1]).reshape(-1,100)
df_tests[2] = np.array(df_tests[2]).reshape(-1,100)

In [47]:
for i in range(3):
    le = LabelEncoder()
    le.fit(list(targets[i]))
    targets[i] = le.transform(targets[i].astype(str))
    clf = LinearSVC().fit(df_trains[i], targets[i])
    predicted = clf.predict(df_tests[i])
    targets_test[i] = le.transform(targets_test[i].astype(str))
    print(classification_report(targets_test[i], predicted))

              precision    recall  f1-score   support

           0       0.43      0.47      0.45       288
           1       0.82      0.79      0.80       865

    accuracy                           0.71      1153
   macro avg       0.62      0.63      0.63      1153
weighted avg       0.72      0.71      0.72      1153

              precision    recall  f1-score   support

           0       0.53      0.87      0.66       124
           1       0.40      0.03      0.05        71
           2       0.63      0.53      0.57        93

    accuracy                           0.55       288
   macro avg       0.52      0.48      0.43       288
weighted avg       0.53      0.55      0.48       288

              precision    recall  f1-score   support

           0       0.85      0.99      0.91       245
           1       0.00      0.00      0.00        43

    accuracy                           0.84       288
   macro avg       0.42      0.49      0.46       288
weighted avg       0